In [ ]:
!python3 -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!python3 -m pip install tokenizers -U
!python3 -m pip install transformers -U

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import jsonlines

/home/rdyro/devel/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the tokenizer and model from Hugging Face

model_id = "mistralai/Mixtral-8x22B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
)

Loading checkpoint shards: 100%|██████████| 59/59 [03:54<00:00,  3.97s/it]


## looping over multiple prompts and logits

In [3]:
# Save to disk
output_path = "golden_data_mixtral-8x22b.jsonl"


# Your prompt text
prompt_texts = ["[INST] I love to [/INST]", "[INST] Today is a [/INST]", "[INST] What is the [/INST]"]
all_data_to_save = []


for prompt_text in prompt_texts:
  # Encode the prompt text
  input_ids = tokenizer.encode(prompt_text, return_tensors="pt")

  # Get the logits for the prompt + completion
  with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

    # Convert logits to fp32
    logits = logits.cpu().numpy().astype("float32")

    # Prepare data to be saved
    data_to_save = {
        "prompt": prompt_text,
        "tokens": input_ids.tolist()[0],
        "logits": logits.tolist()[0],  # Convert numpy array to list for JSON serialization
    }
    all_data_to_save.append(data_to_save)

with jsonlines.open(output_path, "w") as f:
  f.write_all(all_data_to_save)


print(f"Data saved to {output_path}")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Data saved to golden_data_mixtral-8x22b.jsonl
